In [51]:
import pandas as pd
import numpy as np
import dateutil.relativedelta as relativedelta
import dateutil.rrule as rrule
import datetime as dt

from janitor import clean_names
from dateutil.parser import parse as dt_parse

Import our merged data frame and remove any columns that we don't. care about right now (like provider name, address, etc.)

In [52]:
df_merged = pd.read_csv("../data/merged_data.csv")
df_merged = df_merged.clean_names()

# Update to datetime from object
df_merged.admitdate = pd.to_datetime(df_merged.admitdate)
df_merged.dischargedate = pd.to_datetime(df_merged.dischargedate)

In [53]:
df_merged = df_merged.drop([
    'providername', 
    'providerpracticeaddress1', 
    'providerpracticeaddress2', 
    'providerpracticecity',
    'lat',
    'lon'],
    axis = 1
)

In [54]:
df_merged.shape

(40418, 9)

In [55]:
# Drop data without a discharge date (569)
df_merged = df_merged[~df_merged['dischargedate'].isnull()]

In [56]:
# Drop data where patient stayed for more than 100 days (max allowed) -- (316)
df_merged['days_admitted'] = (df_merged['dischargedate'] - df_merged['admitdate']).astype('timedelta64[D]').astype(int)
df_merged = df_merged.loc[df_merged['days_admitted'] <= 100]

In [57]:
# Drop any with less than 3 days (1819, ~4.6%)
# df_merged = df_merged[
#     ((df_merged['dischargedate'] - df_merged['admitdate']).astype('timedelta64[D]').astype(int) >= 3)
# ]

In [58]:
df_merged.shape

(39533, 10)

# Updating Counties
---

In the merged data there are a number of missing or N/A counties. Fortunately there were only 60 or so and we were able to manually associate them with a County after some googling.

In [59]:
county_update = {
    77640: 'Jefferson',
    78526: 'Cameron',
    75707: 'Smith',
    75020: 'Grayson',
    77479: 'Fort Bend',
    75231: 'Dallas',
    79934: 'El Paso',
    79936: 'El Paso',
    76036: 'Tarrant',
    77070: 'Harris',
    76012: 'Tarrant',
    75075: 'Collin',
    79927: 'El Paso',
    77845: 'Brazos',
    77598: 'Harris',
    76132: 'Tarrant',
    75150: 'Dallas',
    76087: 'Parker',
    76051: 'Tarrant',
    78516: 'Hidalgo',
    79703: 'Midland',
    75080: 'Collin',
    77396: 'Harris',
    78572: 'Hidalgo',
    78251: 'Bexar',
    77584: 'Fort Bend',
    77338: 'Harris',
    78229: 'Bexar',
    77573: 'Galveston',
    78754: 'Travis',
    79911: 'El Paso',
    77386: 'Montgomery',
    77074: 'Harris',
    75149: 'Dallas',
    77521: 'Harris',
    78624: 'Gillespie',
    78613: 'Williamson',
    75032: 'Rockwall',
    75862: 'Trinity',
    75703: 'Smith',
    77520: 'Harris',
    75605: 'Gregg',
    78717: 'Williamson',
    78028: 'Kerr',
    78154: 'Guadalupe',
    77340: 'Walker',
    75948: 'Sabine',
    78665: 'Williamson',
    77429: 'Harris',
    75044: 'Dallas',
    78227: 'Bexar',
    76903: 'Tom Green',
    77047: 'Harris',
    77049: 'Harris',
    75038: 'Dallas',
    78233: 'Bexar',
    76711: 'McLennan',
    76109: 'Tarrant'
}

In [60]:
for zipcode, county in county_update.items():
    df_merged.loc[df_merged['providerpracticezipcode'] == zipcode, ['county']] = county

# Feature Engineering
---

Adding a few 'features' to make life easier to transform into time-series data

1. Number of weeks admitted (rounded up to next integer)
2. Start of the admitted and discharged week. Start = Monday of that week.
3. Number of days spent during admit week.
4. Number of days spent during discharge week.
5. Length of stay (days)
6. Discharge days + admit days

In [61]:
# 1. Weeks admitted (rounded)
df_merged['weeks_admitted_rounded'] = np.ceil((df_merged['days_admitted']/7).values)

In [62]:
# 2. Start (Monday) of the admitted or discharged week.
# This will allow the data to be more easily transformed into time series

df_merged['admit_week_start'] = [x - dt.timedelta(days=x.weekday()) for x in df_merged['admitdate']]
df_merged['discharge_week_start'] = [x - dt.timedelta(days=x.weekday()) for x in df_merged['dischargedate']]

In [63]:
# 3. Number of days 'worked' in the admission week (e.g. Tues = 6, Sun = 1)
df_merged['admit_week_days'] = \
    7 - (df_merged['admitdate'] - df_merged['admit_week_start']).astype('timedelta64[D]').astype(int)

In [64]:
# 4. Number of days 'worked' in the discharge week (e.g. Tues= 2, Sun = 7)
df_merged['discharge_week_days'] = \
    (df_merged['dischargedate'] - df_merged['discharge_week_start']).astype('timedelta64[D]').astype(int) + 1

In [65]:
# 5. Length of stay (days)
df_merged['len_of_stay'] = \
    (df_merged['dischargedate'] - df_merged['admitdate']).astype('timedelta64[D]').astype(int)

In [66]:
# 6. Discharge + Admit Days (used in factor calculations)
df_merged['discharge_plus_admit'] = df_merged['discharge_week_days'] + df_merged['admit_week_days']

In [68]:
# 7. Number of days for 'regular' tasks in the first week

df_merged['first_week_regular'] = df_merged['admit_week_days'] - 3

df_merged['first_week_regular'] = np.where(
    df_merged['first_week_regular'] < 0,
    0,
    df_merged['first_week_regular']
)

In [69]:
df_merged.shape

(39533, 18)

# Time Series

Weeks starting on Mondays (more typical for staffing)

In [70]:
# First week in data
before = df_merged['admit_week_start'].min()

# Last week in data
after = df_merged['discharge_week_start'].max()

rr = rrule.rrule(
    rrule.WEEKLY,
    byweekday = relativedelta.MO,
    dtstart = before)

list_of_mondays = rr.between(before,after,inc=True)

In [71]:
# Columns to keep when transformed to time-series compatible format
df_columns_keep = [
    'week',
    'pid', 
    'episodeid', 
    'contracttype', 
    'county',
    'providerpracticezipcode',
    'admit_week_start', 
    'discharge_week_start',
    'first_week_regular',
    'len_of_stay',
    'admitdate',
    'dischargedate',
    'admit_week_days', 
    'discharge_week_days',
    'discharge_plus_admit'
]

Our 'merged data' contains one record for each episode, or patient visit, regardless of how long they may have been in the Skilled Nursing Facility (SNF). MORE DETAILS GO HERE.

To accomplish this, below creates a new dataset with a row for each week a patient (episode) was in a SNF. We use the admitted week (`admit_week_start`) and discharge week (`discharge_week_start`) as bounds for the patient's stay.


The result of this is a DataFrame around 148k rows from our 37.7k patient visits (episodes)

In [72]:
df_out = pd.DataFrame()

for i in list_of_mondays:
    df_tmp = df_merged.loc[
        (i >= df_merged['admit_week_start']) & 
        (i <= df_merged['discharge_week_start'])
    ].copy()
    
    # Add week information
    df_tmp['week'] = i
    
    df_out = pd.concat([df_out, df_tmp])

# Subet of columns
df_timeseries = df_out[df_columns_keep]

# Cleanup memory
del df_out, df_tmp

In [73]:
df_timeseries[df_timeseries['episodeid'] == 17500]

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,first_week_regular,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit
17,2018-12-31,6,17500.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11
17,2019-01-07,6,17500.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11
17,2019-01-14,6,17500.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11


In [74]:
df_timeseries.head()

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,first_week_regular,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit
17,2018-12-31,6,17500.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11
18,2018-12-31,6,17793.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,16,2019-01-01,2019-01-17,6,4,10
19,2018-12-31,6,18516.0,CT2,Brazos,77845,2018-12-31,2019-01-28,3,30,2019-01-01,2019-01-31,6,4,10
20,2018-12-31,6,19016.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,15,2019-01-01,2019-01-16,6,3,9
21,2018-12-31,6,19129.0,CT2,Brazos,77845,2018-12-31,2019-01-07,3,7,2019-01-01,2019-01-08,6,2,8


---
## Creating Factors

Great! Now we have a record for each week a given patient was in a SNF and can take the next step of determining how much work a CC needed to perform for each patient.

_Task Time Estimates_ breaks down the work for each contract type into three categories:

* Admission Tasks -- must be done within 3 days of admission
* Repeated Weekly Tasks -- must be done on a weekly basis
* Discharge Tasks -- must be done during the week of discharge

An important question to ask is _What does weekly mean?_. A patient could, and does in the data, arrive on any day of the week. It didn't make sense for us to force tasks into calendar weeks but **weeks relative to when the patient was admitted**. However, to make timeseries work we need consistent time intervals.

To try and make this compatible, we added 'factors' for each of the three categories of work. The intention is to weight the amount of work required vs what could have been performed relative to when the patient was admitted.

As a quick example, lets use the admission tasks for a patient admitted on a Sunday.

* Week 0: Admit factor = 0.33 (1/3 of admission tasks -- only had 1 day) 
* Week 1: Admit factor = 0.67 (2/3 of admission tasks -- picked up remaining work)

In [79]:
# Creating empty columns for each of our 'factors'
df_timeseries['admit'] = np.nan
df_timeseries['regular'] = np.nan
df_timeseries['discharge'] = np.nan

In [80]:
df_timeseries.head()

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,first_week_regular,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit,admit,regular,discharge
17,2018-12-31,6,17500.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11,NaN,NaN,NaN
18,2018-12-31,6,17793.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,16,2019-01-01,2019-01-17,6,4,10,NaN,NaN,NaN
19,2018-12-31,6,18516.0,CT2,Brazos,77845,2018-12-31,2019-01-28,3,30,2019-01-01,2019-01-31,6,4,10,NaN,NaN,NaN
20,2018-12-31,6,19016.0,CT2,Brazos,77845,2018-12-31,2019-01-14,3,15,2019-01-01,2019-01-16,6,3,9,NaN,NaN,NaN
21,2018-12-31,6,19129.0,CT2,Brazos,77845,2018-12-31,2019-01-07,3,7,2019-01-01,2019-01-08,6,2,8,NaN,NaN,NaN


In [81]:
## ADMIT WEEK EVENTS
## What work was required during the week of admission

'''
1. Three (3) or more days in admit week

Set admit factor to 1.0

Able to perform all admission tasks in that week
'''
df_timeseries['admit'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_days'] >= 3)
    ),
    1, # 100%
    df_timeseries['admit']
)


'''
2. Less than three (3) days in admit week

Create a fractional admit factor (1 = 0.33, 2 = 0.67).
Next week (#3) will pick up the remainder

Could only perform some of the admission tasks in that week
'''
df_timeseries['admit'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_days'] < 3)
    ),
    df_timeseries['admit_week_days'] / 3, # Fraction of 3
    df_timeseries['admit']
)

'''
3. Repeated Weekly Work

Factor = Number of days in the SNF after admission
Examples:
    Tues: 6/7 = 0.857
    Fri:  3/7 = 0.429  
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start'])
    ),
    df_timeseries['first_week_regular'] / 7,
    df_timeseries['regular']
)


'''
4. Admit Week == Discharge Week

Set admit factor to 1.0

If a patient is discharged in the same week as admitted, still have
to perform admission tasks
'''
# 
df_timeseries['admit'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] == df_timeseries['discharge_week_start'])
    ),
    1,
    df_timeseries['admit']
)


'''
5. Discharge Work (Discharge week != admit week)

Set to 0. Discharge did not occur in this week
'''
# 4. Discharge Work Factor -- 0
df_timeseries['discharge'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_days'] >= 3)
    ),
    0,
    df_timeseries['discharge']
)


In [82]:
## REGULAR WEEK EVENTS

'''
1. Admit Work 

Requires previous week to be admission week with less than 3 days.
This is where we "pick up the remainder" from the example
'''
df_timeseries['admit'] = np.where(
    (        
        # Week after admit
        ((df_timeseries['week'] - df_timeseries['admit_week_start']).astype('timedelta64[D]').astype(int) == 7) &
        (df_timeseries['admit_week_days'] < 3)
    ),
    (3 - df_timeseries['admit_week_days']) / 3,
    df_timeseries['admit']
)


'''
2a. Weekly Work (Full Week)

Set 'regular' factor to 1

Perform all the of necessary tasks in that week
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] != df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start'])
    ),
    1,
    df_timeseries['regular']
)

'''
2b. Weekly Work (Not Full Week - Remaining Admit Tasks)
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] != df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_days'] < 3)
    ),
    (7 - (3 - df_timeseries['admit_week_days'])) / 7,
    df_timeseries['regular']
)



'''
3. Weekly Work -- Admit the same week as discharge

For a patient to be discharged all 'weekly tasks' still have
to be performed
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['admit_week_start'] == df_timeseries['discharge_week_start'])
    ),
    1,
    df_timeseries['regular']
)


'''
4. Discharge Work

Set 'discharge' factor to 0

No discharge work in non-discharge week
'''
df_timeseries['discharge'] = np.where(
    (
        (df_timeseries['week'] != df_timeseries['admit_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start'])
    ),
    0,
    df_timeseries['discharge']
)

In [83]:
## DISCHARGE WEEK (admit week != discharge week)

# 1. Admit Work -- Set to 0
'''
1. Admit Work (non-admit week)

Admit factor = 0 if admit value has not already been set
This is done to avoid conflicts with Regular Week logic (#1)
where we had to account for the remaining admit work in the next week
'''
df_timeseries['admit'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        ((df_timeseries['admit'] == 0) | (df_timeseries['admit'].isna()))
    ),
    0,
    df_timeseries['admit']
)


'''
2. Discharge Work

Discharge activites must occur during a discharge week
'''
df_timeseries['discharge'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['discharge_week_start'])
    ),
    1,
    df_timeseries['discharge']
)


'''
3. Regular Work (prorated -- more than a full week)

Need to do the remaining work not done during the admit week.
Remeber we prorated those and have to make up the difference at some point.

Were doing that here!

NOTE - There are a few permutations of this below.
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        ((df_timeseries['discharge_plus_admit'] >= 7) & (df_timeseries['discharge_plus_admit'] <= 9))
    ),
    (7 - df_timeseries['admit_week_days']) / 7, # Math here
    df_timeseries['regular']
)


'''
4. Regular Work (prorated + full week)

Similar to above but what if someone checks in on a Tuesday and leaves on a Saturday
in a different week?

Prorating would only account for 1/7 of a normal weeks worth of work
in a week when the patient was in the SNF for 5 additional days (Tues -> Sat = 5).

Because all weekly tasks need to be done before the patient discharges,
were adding a whole weeks worth of tasks on top of the prorated amount.

Yes, under this scenario 7 days worth of work is being compressed into a shorter timeline
but that must be done in order to discharge the patient (following the flow).

To provide some buffer, were only adding this additional week if 3 or more days past
the proration point.

Example 1 (outside buffer):
    Admitted on Tuesday. Discharged on Saturday (different week).
    
    Prorated admit week = 1 (Monday... Tuesday would be the 8th day of work)
    Days after prorate = 6 (Tues through Sun)
    
    Calculated Factor = 1/7 (prorated) + 1 (next cycle of weekly work. all needs to be done)
    
Example 2 (within buffer):
    Admitted on Tuesday. Discharged on Wednesday (different week).
    
    Prorated admit week = 1 (Monday... Tuesday would be the 8th day of work)
    Days after prorate = 2 (Tues through Wed)
    
    Calculated Factor = 1/7
    Here we are within the 2-day buffer so only account for the prorated amount
'''
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['discharge_week_start']) &
        (df_timeseries['admit_week_start'] != df_timeseries['discharge_week_start']) &
        (df_timeseries['discharge_plus_admit'] > 9)
    ),
    (7 - df_timeseries['admit_week_days']) / 7 + 1, # Math here
    df_timeseries['regular']
)


'''
5. Regular Work (prorated -- less than full week)

Similar to the one above, we're using the same logic but the other way around.
    In #4, the admit day of week (e.g. Tues) was before the discharge (e.g. Sat)
    so we needed to add time,
    
    Here the discharge day of week (e.g. Tues) is before the admit day of week (e.g. Sat).

To accomplish this we prorate the amount of work for the discharge week (like #3)
then back-update the week before discharge and add whatever is remaining
'''
# Week of Discharge (proportional work to # of days)
df_timeseries['regular'] = np.where(
    (
        (df_timeseries['week'] == df_timeseries['discharge_week_start']) &
        (df_timeseries['discharge_plus_admit'] < 7)
    ),
    df_timeseries['discharge_week_days'] / 7,
    df_timeseries['regular']
)

# Week Before
df_timeseries['regular'] = np.where(
    (
        ((df_timeseries['week'] - df_timeseries['discharge_week_start']).astype('timedelta64[D]').astype(int) == -7) &
        (df_timeseries['discharge_plus_admit'] < 7)
    ),
    df_timeseries['regular'] + (7-df_timeseries['discharge_plus_admit']) / 7,
    df_timeseries['regular']
)

In [84]:
# If in SNF < 3 days, removing 'regular' work
df_timeseries['regular'] = np.where(
    (df_timeseries['len_of_stay'] < 3),
    0,
    df_timeseries['regular']
)

In [85]:
# Update any NAs to 0 (shouldn't be any for regular, but doing just in case)
df_timeseries.loc[df_timeseries['admit'].isna(), 'admit'] = 0
df_timeseries.loc[df_timeseries['regular'].isna(), 'regular'] = 0
df_timeseries.loc[df_timeseries['discharge'].isna(), 'discharge'] = 0

In [86]:
# CHECKING THE RESULTS (Admit)
len(df_timeseries['episodeid'].unique()) == df_timeseries['admit'].sum()

True

## Feature Engineering (Part 2)
Adding a few features that might make some interesting visualizations

1. `week_admit` (1, 0) -- Show the change in patient admissions over time
2. `week_discharge` (1, 0) -- Show the change in patient discharges over time




In [87]:
## Setting up columns

df_timeseries['week_admit'] = 0
df_timeseries['week_discharge'] = 0

In [88]:
df_timeseries.loc[df_timeseries['week'] == df_timeseries['admit_week_start'], ['week_admit']] = 1
df_timeseries.loc[df_timeseries['week'] == df_timeseries['discharge_week_start'], ['week_discharge']] = 1

In [89]:
# Double Checking -- Expect them to be 39,532

print(f"Week Admit Records: {df_timeseries['week_admit'].sum()}")
print(f"Week Discharge Records: {df_timeseries['week_discharge'].sum()}")

Week Admit Records: 39532
Week Discharge Records: 39532


In [90]:
df_timeseries['episodeid'] = df_timeseries['episodeid'].astype(int)

# Output to CSV

In [ ]:
# df_timeseries.to_csv('../data/timeseries_base.csv', index=False)

In [91]:
df_timeseries.head()

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,first_week_regular,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit,admit,regular,discharge,week_admit,week_discharge
17,2018-12-31,6,17500,CT2,Brazos,77845,2018-12-31,2019-01-14,3,17,2019-01-01,2019-01-18,6,5,11,1.0,0.428571,0.0,1,0
18,2018-12-31,6,17793,CT2,Brazos,77845,2018-12-31,2019-01-14,3,16,2019-01-01,2019-01-17,6,4,10,1.0,0.428571,0.0,1,0
19,2018-12-31,6,18516,CT2,Brazos,77845,2018-12-31,2019-01-28,3,30,2019-01-01,2019-01-31,6,4,10,1.0,0.428571,0.0,1,0
20,2018-12-31,6,19016,CT2,Brazos,77845,2018-12-31,2019-01-14,3,15,2019-01-01,2019-01-16,6,3,9,1.0,0.428571,0.0,1,0
21,2018-12-31,6,19129,CT2,Brazos,77845,2018-12-31,2019-01-07,3,7,2019-01-01,2019-01-08,6,2,8,1.0,0.428571,0.0,1,0


In [50]:
df_timeseries[df_timeseries['episodeid']==17500]

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit,admit,regular,discharge,week_admit,week_discharge
17,2018-12-31,6,17500,CT2,Brazos,77845,2018-12-31,2019-01-14,17,2019-01-01,2019-01-18,6,5,11,1.0,0.857143,0.0,1,0
17,2019-01-07,6,17500,CT2,Brazos,77845,2018-12-31,2019-01-14,17,2019-01-01,2019-01-18,6,5,11,0.0,1.000000,0.0,0,0
17,2019-01-14,6,17500,CT2,Brazos,77845,2018-12-31,2019-01-14,17,2019-01-01,2019-01-18,6,5,11,0.0,1.142857,1.0,0,1


In [92]:
df_timeseries[df_timeseries['first_week_regular'] == 0]

,week,pid,episodeid,contracttype,county,providerpracticezipcode,admit_week_start,discharge_week_start,first_week_regular,len_of_stay,admitdate,dischargedate,admit_week_days,discharge_week_days,discharge_plus_admit,admit,regular,discharge,week_admit,week_discharge
13,2019-01-28,6,16074,CT2,Brazos,77845,2019-01-28,2019-02-04,0,4,2019-02-01,2019-02-05,3,2,5,1.0,0.285714,0.0,1,0
14,2019-01-28,6,17177,CT2,Brazos,77845,2019-01-28,2019-02-18,0,17,2019-02-01,2019-02-18,3,1,4,1.0,0.000000,0.0,1,0
15,2019-01-28,6,17328,CT2,Brazos,77845,2019-01-28,2019-02-11,0,14,2019-02-01,2019-02-15,3,5,8,1.0,0.000000,0.0,1,0
16,2019-01-28,6,17331,CT2,Brazos,77845,2019-01-28,2019-02-04,0,4,2019-02-01,2019-02-05,3,2,5,1.0,0.285714,0.0,1,0
65,2019-01-28,6,47000,CT2,Brazos,77845,2019-01-28,2019-02-11,0,13,2019-02-01,2019-02-14,3,4,7,1.0,0.000000,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38551,2020-02-24,1731,83,CT1,Harris,77082,2019-11-25,2020-02-24,0,91,2019-12-01,2020-03-01,1,7,8,0.0,0.857143,1.0,0,1
38815,2020-02-24,1749,638,CT1,Gregg,75601,2019-11-25,2020-02-24,0,91,2019-12-01,2020-03-01,1,7,8,0.0,0.857143,1.0,0,1
8728,2020-03-02,415,54202,CT1,Tyler,75979,2019-11-25,2020-03-09,0,100,2019-12-01,2020-03-10,1,2,3,0.0,1.285714,0.0,0,0
28791,2020-03-02,1372,35247,CT2,Rains,75440,2019-11-25,2020-03-02,0,95,2019-12-01,2020-03-05,1,4,5,0.0,0.571429,1.0,0,1
